In [1]:
import os
import csv
import numpy as np

In [2]:
main_dir ="Results_union"
old_drifts = ["sea_P(y)","sea_P(y_x)","sea_P(x_y)","sine_P(y)","sine_P(y_x)","sine_P(x_y)"]
new_drifts = ["jitter","disappearing-minority","appearing-minority","appearing-clusters",
             "shapeshift","minority-share","splitting-clusters","clusters-movement","borderline"]
type_drift = ["incremental","sudden","periodic"]
unbalance_lvls = [1,2,3,4]
stats_w = ["Recall_0","Recall_1", "F1_0", "F1_1", "Gmean"]
stats_tot =["time", "memory"]
algs = ["RebalanceStream","CSMOTE","improved_UOB","improved_OOB","WEOB1","WEOB2","ESOS_ELM","UOB","OOB"]

In [3]:
def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r") as scraped:
        reader = csv.reader(scraped, delimiter=',')
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                row_index += 1
                columns = [str(row_index)]+ [row[i] for i in range(len(row))]
                data.append(columns)
        #print(f"{row_index} valid rows")
        
    return data

# MEAN ALL ROWS

# OLD

In [4]:
final_dict = {}

for old in old_drifts:
    path = main_dir + "/" + old[old.index("P"):]   #old = sea_P(y)
    if old not in final_dict:
        final_dict[old] = {}
        
    for ds in os.listdir(path):
        #check if ds in old (directory giusta)
        if old[:3] == ds[:3]:
            path1 = path + "/" + ds
            
            unbalance = 0
            for u in unbalance_lvls:
                if str(u) in ds:
                    unbalance = u
            assert unbalance != 0
                        
            if unbalance not in final_dict[old]:
                final_dict[old][unbalance] = {}
                
            dtype = "sudden"
            if "G" in ds:
                dtype = "incremental"
                
            if dtype not in final_dict[old][unbalance]:
                final_dict[old][unbalance][dtype] = {}

            for alg in os.listdir(path1):
                if "DS_Store" in alg:
                    continue
                path2 = path1 + "/" + alg
                class0_first = []
                class1_first = []
                class0_second = []
                class1_second = []
                
                print(f"{old} {unbalance} {dtype} {alg}")
                #prendo i valori di tutti i 10 esperimenti
                for result in os.listdir(path2):
                    if "err" not in result or "DS_Store" in result:
                        continue
                    start = False
                    first_half = True
                    data = import_csv(path2+"/"+ result)
                    daily0 = []
                    daily1 = []
                    for data_row in data:
                        if data_row[1] == "learning evaluation instances":
                            start = True
                            continue
                        if not start:
                            continue
                        if float(data_row[1]) > 50000 :
                            first_half = False
                            class0_first.append(np.mean(daily0))
                            class1_first.append(np.mean(daily1))
                            daily0 = []
                            daily1 = []
                        
                        try:
                            daily0.append(float(data_row[11]))
                        except:
                            if data_row[11] == "?":
                                daily0.append(0)
                            else:
                                print("ZERO")
                                print(data_row)
                        try:
                            daily1.append(float(data_row[12]))
                        except:
                            if data_row[12] == "?":
                                daily1.append(0)
                            else:
                                print("ONE")
                                print(data_row)
                    if len(daily0)>0 and len(daily1)>0:
                        class0_second.append(np.mean(daily0))
                        class1_second.append(np.mean(daily1))
                            
                        
                    
                assert len(class0_first) > 0
                assert len(class1_first) > 0
                assert len(class0_second) > 0
                assert len(class1_second) > 0
                
                final_dict[old][unbalance][dtype][alg] = {}
                final_dict[old][unbalance][dtype][alg]["class0_first"] = np.mean(class0_first)
                final_dict[old][unbalance][dtype][alg]["class1_first"] = np.mean(class1_first)

                final_dict[old][unbalance][dtype][alg]["class0_second"] = np.mean(class0_second)
                final_dict[old][unbalance][dtype][alg]["class1_second"] = np.mean(class1_second)


                print(np.mean(class0_first))
                print(np.mean(class1_first))

                print(np.mean(class0_second))
                print(np.mean(class1_second))
                    
                    
                
        

sea_P(y) 1 sudden improved_OOB
99.33238485317229
99.67944534251676
99.64727805137761
99.73587972368956
sea_P(y) 1 sudden WEOB1
99.20190138386944
99.86774740613674
99.46980788854873
99.95936611133686
sea_P(y) 1 sudden OOB
99.30223420307499
99.63319936478649
99.72048449354453
99.4920763917107
sea_P(y) 1 sudden ESOS_ELM
96.8472509479313
94.0009759552319
96.96592572873686
98.59406745225519
sea_P(y) 1 sudden improved_UOB
98.84370737711079
99.78861490357875
99.0239141044412
99.93904916700527
sea_P(y) 1 sudden WEOB2
98.54665893805844
99.71177423661322
98.77101912241004
99.89841527834213
sea_P(y) 1 sudden UOB
99.30223420307499
99.63319936478649
99.72048449354453
99.4920763917107
sea_P(y) 1 sudden RebalanceStream
99.44594648734267
83.63553958686906
99.75154177203957
88.94758228362454
sea_P(y) 3 incremental improved_OOB
94.78578377984928
74.29959131244303
95.38919500918361
74.23632841603339
sea_P(y) 3 incremental WEOB1
94.43030341557585
74.52902973329503
95.02185065551967
74.40023841454328
sea_P

76.74336552037472
81.67680331965623
74.99375780274656
89.49748743718592
sea_P(y_x) 2 sudden RebalanceStream
94.81451459181619
24.94797516475286
95.80274656679151
16.42211055276382
sea_P(y_x) 4 incremental improved_OOB
78.38427651686746
84.98155996462506
75.44473500055247
91.41688610006163
sea_P(y_x) 4 incremental WEOB1
78.3544917316806
84.96548716621984
75.36738978306508
91.47291164771136
sea_P(y_x) 4 incremental OOB
76.57749806305328
88.82382467299117
74.8664874221944
93.28813939156257
sea_P(y_x) 4 incremental ESOS_ELM
79.1156550002613
77.19565754214027
76.76512835622997
83.71953610846546
sea_P(y_x) 4 incremental improved_UOB
77.9853871229307
84.9083541822032
75.17955139773858
91.56255252395093
sea_P(y_x) 4 incremental WEOB2
85.45150834660932
61.39415217491627
79.78711649662998
76.85024371113226
sea_P(y_x) 4 incremental UOB
77.02230511092377
85.42382898354701
75.51471400685058
90.37481091377667
sea_P(y_x) 4 incremental RebalanceStream
79.54425440601281
80.06426479731766
76.58281462929

81.36905163255491
86.48469064814596
81.72312601498618
87.3028655794913
sea_P(x_y) 3 sudden UOB
78.56455217327306
88.86061164898747
79.2814610102852
89.29602040131535
sea_P(x_y) 3 sudden RebalanceStream
91.37796174709626
68.62397205783729
91.64648565486196
69.56580095295617
sea_P(x_y) 2 incremental improved_OOB
95.03515851404953
75.87679708686535
96.70664487715702
75.57603686635944
sea_P(x_y) 2 incremental WEOB1
92.76254526627054
78.41312444740241
94.05589563194194
78.34101382488481
sea_P(x_y) 2 incremental OOB
93.0105304140808
78.12365476819993
94.01988754397141
78.66696639316623
sea_P(x_y) 2 incremental ESOS_ELM
91.97671165111973
75.26051084810425
92.23776417472232
79.33685511970329
sea_P(x_y) 2 incremental improved_UOB
88.65564491834886
80.18772129219302
90.95366036063483
80.69012026525795
sea_P(x_y) 2 incremental WEOB2
89.07052250849567
77.2412868470035
95.75935517824004
75.3512419916826
sea_P(x_y) 2 incremental UOB
85.18522895889426
82.97832191062493
89.0092236102263
82.44352028773

97.84526004927578
96.10803345535534
99.12248212199778
97.86591503925911
sine_P(y_x) 3 sudden WEOB1
97.1178236412283
96.15930320620024
98.80623379583463
97.98671230118784
sine_P(y_x) 3 sudden OOB
97.90547233766148
97.05287926519013
98.93729166073106
98.63096436480772
sine_P(y_x) 3 sudden ESOS_ELM
90.47597678649824
83.39636509027014
96.1112852218012
93.10046305617074
sine_P(y_x) 3 sudden improved_UOB
95.57481427233111
96.07285623345491
98.00849027037806
97.9464465472116
sine_P(y_x) 3 sudden WEOB2
67.62577276256363
66.26930583139836
79.06777970882362
80.34360110059728
sine_P(y_x) 3 sudden UOB
94.58547030439878
95.16463501604956
97.04265078777175
97.98671230118784
sine_P(y_x) 3 sudden RebalanceStream
97.32388146822665
93.90569138290306
98.78344112367873
95.91973693040735
sine_P(y_x) 3 incremental improved_OOB
95.02644644860771
91.52975555602852
98.94430747834882
97.59766502020656
sine_P(y_x) 3 incremental WEOB1
94.10721436403452
92.17964575021504
98.70409001833238
97.90450531357581
sine_P(

97.88207058450344
93.92002667894262
97.77526745963186
98.72223897095769
sine_P(x_y) 4 sudden improved_UOB
99.06061002471974
99.62450821360198
99.36872881919065
99.74374434730177
sine_P(x_y) 4 sudden WEOB2
99.03630973376623
99.56768225359124
99.27569938201873
99.66335041704352
sine_P(x_y) 4 sudden UOB
99.70518864477033
99.41765860933472
99.79732872616123
99.56285800422066
sine_P(x_y) 4 sudden RebalanceStream
99.11206955103235
99.13876844203885
99.4252109774736
99.326700834087
sine_P(x_y) 1 sudden improved_OOB
99.82249769043578
98.11422855050645
99.92235680376237
98.55749695245835
sine_P(x_y) 1 sudden WEOB1
99.53118009824436
98.81990393766135
99.7892541816407
98.94351889475823
sine_P(x_y) 1 sudden OOB
99.7999107073264
98.38103355128946
99.92013842672701
98.78098334010565
sine_P(x_y) 1 sudden ESOS_ELM
97.53591913976157
88.73913214289068
98.22219264386175
92.67370987403494
sine_P(x_y) 1 sudden improved_UOB
98.27140814597632
98.81800630117758
99.16367185766894
98.7403494514425
sine_P(x_y) 1

In [5]:
old = final_dict.copy()

# NEW

In [6]:
final_dict = old.copy()

In [7]:
for new in new_drifts:
    path = main_dir + "/" + new 
    
    if new not in final_dict:
        final_dict[new] = {}
        
    for unbalance in range(1,5):
        if unbalance not in final_dict[new]:
            final_dict[new][unbalance] = {}


        path1 = path + "/" + str(unbalance)

        for alg in os.listdir(path1):
            if "DS_Store" in alg:
                continue
            path2 = path1 + "/" + alg
     
            
            alg_dtypes = {}
            alg_dtypes["incremental"] = {}
            alg_dtypes["sudden"] = {}
            alg_dtypes["periodic"] = {}
            alg_dtypes["incremental"]["class0_first"] = []
            alg_dtypes["sudden"]["class0_first"] = []
            alg_dtypes["periodic"]["class0_first"] = []
            alg_dtypes["incremental"]["class0_second"] = []
            alg_dtypes["sudden"]["class0_second"] = []
            alg_dtypes["periodic"]["class0_second"] = []
            alg_dtypes["incremental"]["class1_first"] = []
            alg_dtypes["sudden"]["class1_first"] = []
            alg_dtypes["periodic"]["class1_first"] = []
            alg_dtypes["incremental"]["class1_second"] = []
            alg_dtypes["sudden"]["class1_second"] = []
            alg_dtypes["periodic"]["class1_second"] = []
            
            first_half = True

            print(f"{new} {unbalance} {alg}")
            #prendo i valori di tutti i 10 esperimenti dei 3 dtype
            
            for result in os.listdir(path2):
                if "err" not in result or "DS_Store" in result:
                    continue
                daily0 = []
                daily1 = []
                data = import_csv(path2+"/"+ result)
                start = False
                
                for data_row in data:
                    if data_row[1] == "learning evaluation instances":
                        start = True
                        continue
                    if not start:
                        continue
                    if float(data_row[1]) > 50000 :
                        first_half = False
                        
                        alg_dtypes[result[:result.index("_")]]["class0_first"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1_first"].append(np.mean(daily1))
                        daily0 = []
                        daily1 = []

                    try:
                        daily0.append(float(data_row[11]))
                    except:
                        if data_row[11] == "?":
                            daily0.append(0.0)
                        else:
                            print("ZERO")
                            print(data_row)
                    try:
                        daily1.append(float(data_row[12]))
                    except:
                        if data_row[12] == "?":
                            daily1.append(0.0)
                        else:
                            print("ONE")
                            print(data_row)
                        
                    
                
                try:
                    if len(daily0)>0 and len(daily1)>0:
                        alg_dtypes[result[:result.index("_")]]["class0_second"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1_second"].append(np.mean(daily1))
                except:
                    print(f"error with {result}")
            for k,i in alg_dtypes.items():      
                
                if k not in final_dict[new][unbalance]:
                    final_dict[new][unbalance][k] = {}
                assert alg not in final_dict[new][unbalance][k]
                final_dict[new][unbalance][k][alg] = {}
                final_dict[new][unbalance][k][alg]["class0_first"] = np.mean(i["class0_first"])
                final_dict[new][unbalance][k][alg]["class1_first"] = np.mean(i["class1_first"])
                final_dict[new][unbalance][k][alg]["class0_second"] = np.mean(i["class0_second"])
                final_dict[new][unbalance][k][alg]["class1_second"] = np.mean(i["class1_second"])
                print(k)
                print(np.mean(i["class0_first"]))
                print(np.mean(i["class1_first"]))
                print(np.mean(i["class0_second"]))
                print(np.mean(i["class1_second"]))

jitter 1 improved_OOB
incremental
88.83957035868445
83.81067584289096
88.84254689042447
87.25446428571426
sudden
97.6635761506687
98.47801956089853
99.65570092627557
99.95984742019675
periodic
88.83957035868445
83.81067584289096
88.84254689042447
87.25446428571426
jitter 1 WEOB1
incremental
83.71874190096364
92.08806151452674
84.40769990128332
95.02232142857143
sudden
96.75726883654228
98.75733209835957
99.44912148204091
99.91969484039349
periodic
83.71874190096364
92.08806151452674
84.40769990128332
95.02232142857143
jitter 1 CSMOTE
incremental
74.54490409352988
83.41253348510847
73.89930898321816
85.80357142857142
sudden
78.36839982894983
97.3150343859763
80.95026544347942
99.2371009837382
periodic
74.54490409352988
83.41253348510847
73.89930898321816
85.80357142857142
jitter 1 OOB
incremental
86.69549555899329
87.52249671683111
87.322309970385
90.89285714285714
sudden
97.20090105173304
98.79344945946052
99.61793909238322
99.9397711302951
periodic
86.69549555899329
87.52249671683111


incremental
77.65004468484923
97.91667495921422
84.09065659371183
98.81091292993277
sudden
93.28138850291406
99.49795105753597
98.5180734955442
99.98003892409801
periodic
77.65004468484923
97.91667495921422
84.09065659371183
98.81091292993277
jitter 4 WEOB2
incremental
81.5090691294335
93.24182049934494
85.03869940376993
97.70517308440296
sudden
78.57164378359434
99.43320939590635
86.87961015987449
99.98003892409801
periodic
81.5090691294335
93.24182049934494
85.03869940376993
97.70517308440296
jitter 4 UOB
incremental
85.26956227816565
95.14936113223594
88.99751879420805
96.59387675723733
sudden
94.80620849575793
99.1965034241544
98.88521744935085
99.98003892409801
periodic
85.26956227816565
95.14936113223594
88.99751879420805
96.59387675723733
jitter 4 RebalanceStream
incremental
81.97475350307019
92.30377807815994
86.24967596193017
95.25476468300272
sudden
95.14732390424389
98.94374676197785
98.99869830780014
99.96506811717151
periodic
81.97475350307019
92.30377807815994
86.24967596

incremental
75.78312552344445
19.574261020532784
75.69111111111111
0.0
sudden
73.38594374851185
9.55432153340417
73.22400000000002
0.0
periodic
81.81938747349373
98.99841563013089
82.99327312595236
99.51362405350137
disappearing-minority 4 WEOB2
incremental
80.2736086119447
19.505192823175584
81.18666666666668
0.0
sudden
72.23967007227157
9.489579871774541
72.12199999999999
0.0
periodic
81.09583838084136
98.8600896904308
82.21652358122422
99.57442104681368
disappearing-minority 4 UOB
incremental
96.64630015427707
18.846357502581007
100.0
0.0
sudden
96.33983393826719
9.252873900022614
99.42600000000002
0.0
periodic
77.0020847420322
96.43532475983967
80.48091574683168
97.68971425413143
disappearing-minority 4 RebalanceStream
incremental
97.5003538110422
13.349196693656049
100.0
0.0
sudden
97.87020100069029
9.037437236541969
99.96000000000001
0.0
periodic
78.71421780754133
91.05545874049443
80.3508380718772
95.73868346874482
appearing-minority 1 improved_OOB
incremental
89.88179137161138


0.0
appearing-minority 4 WEOB2
incremental
78.59163883889524
89.55273618931584
79.29435334273207
99.12445971406406
sudden
73.69852188883786
86.0326354577125
74.55075767735015
98.21039064049178
periodic
70.71223131543489
10.182133364151161
66.93333333333335
0.0
appearing-minority 4 UOB
incremental
81.2128832169058
89.30646971940179
79.59857535059732
99.6785991355425
sudden
75.87534309199141
87.35511343363397
77.26632694112914
98.97382510410469
periodic
71.89306098572435
9.839824338379753
67.75333333333333
0.0
appearing-minority 4 RebalanceStream
incremental
85.47214002007117
81.20634127100077
83.25294946946651
94.62484761165909
sudden
80.88140629554981
79.22574781956942
81.11841223011935
92.67796946262146
periodic
98.83220850611673
6.349955699372086
100.0
0.0
appearing-clusters 1 improved_OOB
incremental
87.6965918241311
86.27459574343247
86.46883395361446
87.734357352609
sudden
88.22501858022511
88.3486159001325
87.36545418229433
89.63772515685085
periodic
93.65064689834615
99.40218547

incremental
84.59209045221822
97.35740567273282
82.03946634954846
98.44674147366094
sudden
83.96237542937214
96.96356555571751
82.82060723730459
98.29085308292342
periodic
92.84056973515966
99.96197048871448
93.10996627256216
99.98335090737555
appearing-clusters 4 CSMOTE
incremental
87.3106772771341
91.67690708335407
84.48474488089485
92.87491017633077
sudden
87.65274279054724
90.47016312154004
85.84005623807452
92.56222984051274
periodic
93.41788220167848
99.6456609529559
93.67703198547125
99.77801209834065
appearing-clusters 4 OOB
incremental
85.19307168519508
97.6433361043861
82.84588799286485
98.43568625283288
sudden
84.31514793218913
97.67824916007288
83.15870518528436
98.65354995776818
periodic
93.6163019063843
99.42978910350604
93.91052963196321
99.5171763138909
appearing-clusters 4 ESOS_ELM
incremental
73.34599390638711
85.91361049216408
71.32706529413949
89.8164833342546
sudden
72.54001792948388
85.8657091284994
71.16292304087304
90.84960500819794
periodic
77.32180997326823
95

incremental
79.4867162089402
95.10598330239677
82.02197102372234
97.29312247149687
sudden
80.78556201360242
95.05893110510333
83.0158957468137
97.40139211136889
periodic
79.72186594154371
94.70206471039756
81.97153567571002
96.6572992160923
shapeshift 3 UOB
incremental
78.74483869498225
96.22331876396753
81.52205062888075
98.22728944464879
sudden
78.83718867780351
95.78553347776382
81.72991550909352
98.01789857474314
periodic
76.59393825048784
97.09704672334568
78.68987864540314
98.8833012867919
shapeshift 3 RebalanceStream
incremental
88.98831941871036
79.65762859932741
89.22464575704507
83.83229128356014
sudden
87.58614578731753
82.3799225567563
88.28583703279392
86.34405038117336
periodic
88.65216671893069
83.26365629895821
89.35446978842366
85.11314894246414
shapeshift 4 improved_OOB
incremental
81.64983751528516
95.1337130942567
83.13530372790731
97.38211202125305
sudden
82.10232143429543
93.63801419543505
83.50804567002737
96.33842163025042
periodic
80.53054526779893
96.656944537

incremental
81.67076519815629
99.07288547313746
82.69188057238952
99.56241192612922
sudden
81.78620405224333
99.20334016088503
82.85934555422307
99.6060889304313
periodic
81.41073990514955
19.612076771711184
82.2222222222222
0.0
minority-share 3 WEOB2
incremental
80.79601225573772
98.8846414821203
82.27623187486118
99.50307795001113
sudden
81.10951103626752
98.88147751688132
82.47672891325453
99.5393243423688
periodic
80.36755558366404
19.4922597495376
81.22666666666667
0.0
minority-share 3 UOB
incremental
80.31601433173878
99.58768927829435
81.83202715994544
99.84424831269006
sudden
80.24775256629809
99.56796432225809
81.81143281365999
99.83976498865002
periodic
79.66482889149862
19.79456631550571
80.87777777777777
0.0
minority-share 3 RebalanceStream
incremental
86.06430068033227
67.63792661367643
85.75372021448742
80.1379514944745
sudden
88.48513899241374
85.42905498403007
88.69853235109359
87.21458138603285
periodic
98.49496903171575
7.794195022349644
100.0
0.0
minority-share 4 imp

incremental
84.50774103643569
96.76718067402767
83.25196550849606
98.55083234244947
sudden
82.26998058209917
96.09445564009442
82.10511282490715
98.24549699799866
periodic
90.08282472728268
98.00878337313789
90.86413491409371
99.45741043555822
splitting-clusters 3 CSMOTE
incremental
83.0088737330323
93.13665448012632
81.61615521176769
95.94976218787158
sudden
82.99065213001946
91.96599651023834
82.62496429591545
94.56971314209474
periodic
89.24011700186594
96.50484549170865
89.95435237431052
98.18641296268767
splitting-clusters 3 OOB
incremental
82.88334834179943
97.68438238763143
81.95219376109561
99.54667063020214
sudden
82.51732123359324
96.69985731738278
82.09654384461581
98.7525016677785
periodic
90.17050319557606
97.43644593151649
90.8134153299943
99.51687230563401
splitting-clusters 3 ESOS_ELM
incremental
74.01107651812177
86.74042407915637
71.89766675120468
90.18356123662308
sudden
73.46027887108308
85.44215336889582
71.89745786918022
90.9092728485657
periodic
77.7773553282473


incremental
80.16039833279919
92.54003178571455
81.72367910406388
97.4792740309209
sudden
79.20690268305616
85.79577053576429
80.70363804339684
94.81459149834187
periodic
72.59265373224186
93.27656578273445
75.00415950307801
98.63504139628552
clusters-movement 2 WEOB2
incremental
75.70845669338568
77.76789301935148
80.68137716915228
89.13287026663679
sudden
74.3403912846324
64.10582781444283
79.62246248345777
83.0770776806351
periodic
75.81159580038354
93.54489611183111
79.13593256059008
98.34414857910045
clusters-movement 2 UOB
incremental
77.65683176114926
89.94243089364254
80.16577036092477
96.19090298005825
sudden
64.49161290123958
88.71456769229236
73.70221478688606
95.81951562657018
periodic
70.01947300621774
94.3826306021727
73.4956463867783
98.88118147236517
clusters-movement 2 RebalanceStream
incremental
95.34929262028513
55.047325006186014
95.2957808948273
61.50571364552992
sudden
95.13295100720515
51.67569465348282
94.94369397488076
64.89800020098482
periodic
93.658909335001

incremental
83.93387659267766
99.24688024237672
85.90965948575399
99.65650969529086
sudden
81.52081347651166
99.39311713734216
84.07927909937347
99.87926350739511
periodic
89.2538575354555
99.45336743285142
93.58382467460228
100.0
borderline 2 CSMOTE
incremental
81.18057949747904
97.63440446490026
81.23974982626825
98.62603878116343
sudden
77.60342692942183
97.3799520858506
77.10491500461796
97.97766374886808
periodic
85.08285653753535
98.2832362402313
86.7532539770831
99.19283502874832
borderline 2 OOB
incremental
84.2223624166431
99.2600012058085
86.0736622654621
99.51246537396123
sudden
82.02251930863868
99.48900637721312
84.46868525498613
99.74846564040648
periodic
90.07892506348054
99.6671927262701
94.75748136611413
100.0
borderline 2 ESOS_ELM
incremental
73.03457953549392
90.59328302387594
71.33620569840167
95.09916897506926
sudden
72.75083138856144
91.26550961669132
71.12728089663264
96.32759834993462
periodic
75.81123636651508
92.10177878216004
73.98737345644676
97.108580274214

In [8]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0_first': 99.33238485317229,
     'class1_first': 99.67944534251676,
     'class0_second': 99.64727805137761,
     'class1_second': 99.73587972368956},
    'WEOB1': {'class0_first': 99.20190138386944,
     'class1_first': 99.86774740613674,
     'class0_second': 99.46980788854873,
     'class1_second': 99.95936611133686},
    'OOB': {'class0_first': 99.30223420307499,
     'class1_first': 99.63319936478649,
     'class0_second': 99.72048449354453,
     'class1_second': 99.4920763917107},
    'ESOS_ELM': {'class0_first': 96.8472509479313,
     'class1_first': 94.0009759552319,
     'class0_second': 96.96592572873686,
     'class1_second': 98.59406745225519},
    'improved_UOB': {'class0_first': 98.84370737711079,
     'class1_first': 99.78861490357875,
     'class0_second': 99.0239141044412,
     'class1_second': 99.93904916700527},
    'WEOB2': {'class0_first': 98.54665893805844,
     'class1_first': 99.71177423661322,
     'class0_se

In [9]:
def mean_of(_dict ):
    values_0_first = []
    values_1_first = []
    values_0_second = []
    values_1_second = []
    for alg, values in _dict.items():
        values_0_first.append(values["class0_first"])
        values_1_first.append(values["class1_first"])
        values_0_second.append(values["class0_second"])
        values_1_second.append(values["class1_second"])
    return np.mean(values_0_first),np.mean(values_1_first),np.mean(values_0_second),np.mean(values_1_second)

In [12]:

with open('RecallXRecall_HALF.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","Recall_0_first", "Recall_1_first","Recall_0_second", "Recall_1_second"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
                m0f,m1f,m0s,m1s = mean_of(algs)
                for alg, values in algs.items():
                    print([drift,unbalance,dtype,alg])
                    print(values)
                    print([m0f,m1f,m0s,m1s])
                    writer.writerow([drift,unbalance,dtype,
                                     alg,values["class0_first"] - m0f,
                                     values["class1_first"] - m1f,
                                     values["class0_second"] - m0s,
                                     values["class1_second"] - m1s])

['sea_P(y)', 1, 'sudden', 'improved_OOB']
{'class0_first': 99.33238485317229, 'class1_first': 99.67944534251676, 'class0_second': 99.64727805137761, 'class1_second': 99.73587972368956}
[98.85278979920436, 96.99381202006492, 99.13380695683038, 98.25731409995936]
['sea_P(y)', 1, 'sudden', 'WEOB1']
{'class0_first': 99.20190138386944, 'class1_first': 99.86774740613674, 'class0_second': 99.46980788854873, 'class1_second': 99.95936611133686}
[98.85278979920436, 96.99381202006492, 99.13380695683038, 98.25731409995936]
['sea_P(y)', 1, 'sudden', 'OOB']
{'class0_first': 99.30223420307499, 'class1_first': 99.63319936478649, 'class0_second': 99.72048449354453, 'class1_second': 99.4920763917107}
[98.85278979920436, 96.99381202006492, 99.13380695683038, 98.25731409995936]
['sea_P(y)', 1, 'sudden', 'ESOS_ELM']
{'class0_first': 96.8472509479313, 'class1_first': 94.0009759552319, 'class0_second': 96.96592572873686, 'class1_second': 98.59406745225519}
[98.85278979920436, 96.99381202006492, 99.1338069568

{'class0_first': 85.02156062172013, 'class1_first': 96.4406587881789, 'class0_second': 86.78711965859335, 'class1_second': 98.59771146511106}
[79.90571064215538, 90.5027430619942, 83.15280779865382, 93.81547129359559]
['jitter', 2, 'incremental', 'ESOS_ELM']
{'class0_first': 74.96533575451376, 'class1_first': 86.28066671129983, 'class0_second': 75.61020894529733, 'class1_second': 92.66771370877271}
[79.90571064215538, 90.5027430619942, 83.15280779865382, 93.81547129359559]
['jitter', 2, 'incremental', 'improved_UOB']
{'class0_first': 67.90470320342068, 'class1_first': 96.83071257353853, 'class0_second': 76.76938424873913, 'class1_second': 99.18106349562484}
[79.90571064215538, 90.5027430619942, 83.15280779865382, 93.81547129359559]
['jitter', 2, 'incremental', 'WEOB2']
{'class0_first': 78.9229299900856, 'class1_first': 93.08025179420416, 'class0_second': 84.10741007592972, 'class1_second': 97.01592999775633}
[79.90571064215538, 90.5027430619942, 83.15280779865382, 93.81547129359559]
['

[84.9050545350584, 89.57124083096572, 83.67360071004605, 91.73507712944334]
['appearing-clusters', 2, 'sudden', 'ESOS_ELM']
{'class0_first': 72.87724596211376, 'class1_first': 85.32216272749923, 'class0_second': 71.51223165252122, 'class1_second': 90.7062374245473}
[84.9050545350584, 89.57124083096572, 83.67360071004605, 91.73507712944334]
['appearing-clusters', 2, 'sudden', 'improved_UOB']
{'class0_first': 84.3246027361975, 'class1_first': 94.18968269582707, 'class0_second': 82.7408886669995, 'class1_second': 97.28370221327967}
[84.9050545350584, 89.57124083096572, 83.67360071004605, 91.73507712944334]
['appearing-clusters', 2, 'sudden', 'WEOB2']
{'class0_first': 83.96483589617849, 'class1_first': 92.61579558591386, 'class0_second': 82.74338492261609, 'class1_second': 96.2877263581489}
[84.9050545350584, 89.57124083096572, 83.67360071004605, 91.73507712944334]
['appearing-clusters', 2, 'sudden', 'UOB']
{'class0_first': 83.3298757189282, 'class1_first': 94.32192147430708, 'class0_secon

{'class0_first': 84.50774103643569, 'class1_first': 96.76718067402767, 'class0_second': 83.25196550849606, 'class1_second': 98.55083234244947}
[82.49753975337974, 94.0001440866688, 81.24221545918223, 96.31316884661118]
['splitting-clusters', 3, 'incremental', 'CSMOTE']
{'class0_first': 83.0088737330323, 'class1_first': 93.13665448012632, 'class0_second': 81.61615521176769, 'class1_second': 95.94976218787158}
[82.49753975337974, 94.0001440866688, 81.24221545918223, 96.31316884661118]
['splitting-clusters', 3, 'incremental', 'OOB']
{'class0_first': 82.88334834179943, 'class1_first': 97.68438238763143, 'class0_second': 81.95219376109561, 'class1_second': 99.54667063020214}
[82.49753975337974, 94.0001440866688, 81.24221545918223, 96.31316884661118]
['splitting-clusters', 3, 'incremental', 'ESOS_ELM']
{'class0_first': 74.01107651812177, 'class1_first': 86.74042407915637, 'class0_second': 71.89766675120468, 'class1_second': 90.18356123662308}
[82.49753975337974, 94.0001440866688, 81.24221545